# Install and Authenticate GDrive Libraries

In [0]:
# authenticate and mount gdrive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# install pydrive and google API
!pip install PyDrive
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [0]:
# import stuff
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from google.colab import files
from oauth2client.client import GoogleCredentials

In [0]:
# authenticate gdrive use for pydrive

from pydrive.drive import GoogleDrive

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# Collect DeOldify Code from GitHub

In [0]:
cd /content

In [0]:
from os import path
import torch

In [0]:
!git clone https://github.com/jantic/DeOldify.git DeOldify 

In [0]:
cd /content/DeOldify

In [0]:
!pip install -r requirements.txt

In [0]:
import fastai
from deoldify.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True

In [0]:
!mkdir 'models'
!wget wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth

# Unzip and Colorize Photos
Given an ID of a GDrive folder of zips of b&w images, iterate through the zips, unzip them, colorize the images inside, zip the colorized images to a new folder, and delete the original zip and all the b&w photos.

In [0]:
colorizer = get_image_colorizer(artistic=True)

In [0]:
# ID of the folder of b&w zips
zips_folder_id = '13Ap3P8G2TX_S6N0rbqFy7V-wiEyWRPcW' #@param {type: "string"}

In [0]:
path_root = "/content/gdrive/My Drive/"

# name of folder above on Google Drive
zip_folder_name = 'test' #@param {type: "string"}

path_to_zips_folder = path_root + zip_folder_name

In [0]:
# path to the folder that will contain the new zips of colorized photos
colorized_zips_folder = "/content/gdrive/\"My Drive\"/colorized" #@param {type: "string"}

In [0]:
!echo "$path_to_zips_folder"

In [0]:
cd $path_to_zips_folder

In [0]:
from deoldify.visualize import *
drive = GoogleDrive(gauth)

# retrieve google file information on all zips
zip_file_list = drive.ListFile(
    {'q': "'%s' in parents" % zips_folder_id}).GetList()

file_info = {}
counter = 0

for zip_file in zip_file_list:
  counter += 1
  file_info[zip_file['title'].split(".")[0]] = zip_file['id']
    
zips_left = len(file_info)

print("zips left:", zips_left)
#print(file_info)
#print(len(file_info))

In [0]:
from deoldify.visualize import *

import time
import subprocess

for box_zip in file_info:
  
  #unzip current zip file into folder
  zip_name = box_zip + ".zip"
  !echo "$zip_name"
  !echo "$box_zip"
  
  zip_path = path_to_zips_folder + "/" + zip_name
  
  if not os.path.exists(zip_path):
    continue
  
  print("\n")
  print("unzipping", box_zip, "\n")
  
  !unzip -q $zip_name -d $box_zip
  
  #wait 10 secs for the new folder to load
  print("loading", box_zip, "folder (10 sec)\n")
  time.sleep(10)
  
  %cd $box_zip
  num_files = int(os.popen("ls | wc -l").read().split('\n')[0])
  print("files unzipped:", num_files, "\n")
  %cd ..
  
  #get id of new b&w folder
  newest_folder = drive.ListFile(
      {'q': "mimeType = 'application/vnd.google-apps.folder' and '%s' in parents" % zips_folder_id}).GetList()[0]
  
  #update dictionary
  file_info[box_zip] = newest_folder['id']
  
  #id is currently that of the folder made from the zip file
  folder_id = file_info[box_zip]
  
  #print("loading images in", box_zip, "folder (20 sec)\n")
  #time.sleep(20)
  
  #retrieve google file information on all images in folder
  file_list = {}
  
  while len(file_list) < num_files:
    time.sleep(1)
    file_list = drive.ListFile(
        {'q': "'%s' in parents" % folder_id}).GetList()
    print("current files loaded in", box_zip + ":", len(file_list))
  
  #print("file_list = ", file_list)
  print("total files loaded in", box_zip + ":", len(file_list), "\n")
  
  image_titles = []
  
  #print(image_titles)
  
  #get list of image titles
  for img in file_list:
      if img['mimeType'] == "image/png":
        image_titles += [img['title']]
        
  print("images: \n", image_titles)
  
  # resolution of colorizer (from 7 to 45)
  render_factor = 35  #@param {type: "slider", min: 7, max: 45}
  
  move_directory = path_to_zips_folder + "/" + box_zip
  !echo "$move_directory"
  %cd $move_directory
  
  #colorize!
  for image in image_titles:

      img_path = path_to_zips_folder + '/' + box_zip + '/' + image

      if img_path is not None and img_path !='':
          img_path = Path(img_path)
          
          #transform and save the image
          clr_img = colorizer.get_transformed_image(img_path, render_factor=render_factor)
          clr_img2 = colorizer._save_result_image(img_path, clr_img)
          
          print(image + " colorized")

      else:
          print('Provide an image url and try again.')
            
  print("\n", box_zip, "is all colorized!\n")
  
  %cd ..
  
  #zip all new colorized images and save to a general folder
  colorized_zip_path = colorized_zips_folder + '/' + "CLR_" + box_zip
  !echo "$colorized_zip_path"
  
  print("\n")
  
  curr_result_images = box_zip + "/result_images"
  !zip -r -q $colorized_zip_path $curr_result_images
 
  print("CLR_" + box_zip, "has been zipped \n")
  
  #reauthenticate
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  local_download_path = os.path.expanduser('~/data')
  try:
    os.makedirs(local_download_path)
  except: pass

  print("authenticated")
  
  #delete the original zip
  for zip_file in zip_file_list:
    if zip_file['title'].split(".")[0] == box_zip:
      zip_file.Delete()
   
  box_folder = drive.CreateFile({'id': folder_id})
  box_folder.Delete()
 
  #!echo "$box_zip"
  #%rm -r $box_zip
  
  print("\n")
  print(box_zip, "zip and folder deleted\n")
  
  zips_left -= 1
  
  print("zips left:", zips_left, "\n")
  
  print("next box!\n")
  print("------------------------------------------------------\n")
  print("\n")
  
  
print("""
      /\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
                        Colorizing complete!
      /\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/""")    